# step 5: analyze useful nouns

import librarys and load data set

In [1]:
# librarys
import pandas as pd
import nltk
import sklearn
import nltk.sentiment

import re
import pandas as pd
import nltk
import sklearn
from nltk.stem import WordNetLemmatizer
import numpy as np
from nltk.tokenize import word_tokenize
import nltk.sentiment
import string
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn import linear_model
import collections
from gensim.models import Word2Vec
from gensim.models import word2vec
import multiprocessing
import time
import inflect
from textblob import TextBlob
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

## define stopwords
sr = stopwords.words('english')
p = inflect.engine()
wnl = WordNetLemmatizer() 
table = str.maketrans('', '', string.punctuation)

Using TensorFlow backend.


In [2]:
# function to change text to words
def clean_review(text):
    #Change n't into not
    x = re.sub(r'n\'t',' not',text)
    #Change not word into not_word
    x = re.sub(r'not ','not_',x)
    #Split into words
    x = word_tokenize(x)
    #Remove punctuation
    x = [w.translate(table) if not re.match(r'not_.*', w) else w for w in x]
    #Change numbers into words
    x = [p.number_to_words(w) if w.isdigit() else w  for w in x ]
    #Remove non alphabetic
    x = [w for w in x if w.isalpha() or re.match(r'not_.*',w)]
    #Convert to lower case
    x = [w.lower() for w in x]
    #Remove stop words
    x = [w for w in x if not w in sr]
    ## lemmatization
    x = [wnl.lemmatize(w) for w in x]
    return x

In [25]:
# load review data
review_bind=pd.read_csv("review_bind.csv")
review_bind.head()

,business_id,stars,bind_text
0,nsNONDHbV7Vudqh21uicqw,3.5,Was very excited for happy hour and heard grea...
1,F06m2yQSPHIrb1IT7heYeQ,4.0,A Cantonese restaurant on the southwest side o...
2,c_FTil8s5PS2l_YJDQAXhA,4.0,"Great food, friendly service, lovely atmospher..."
3,vAbRNF0hJt91oCUpm_sdKw,3.5,I'm a FAN. Loved this place. Great service a...
4,z8H4Mdzl4jS9pYswj6Jf9w,3.5,Service was really poor.\r\r\nMay try the King...


In [29]:
# load dictionary of adjectives with positive/negative tags
f = open('dict_adj.txt','r')
a = f.read()
dict_adj = eval(a)
f.close()

do word tokenization

In [5]:
# do word tokenization
clean_text = [[1]]*len(review_bind)
for i in range(len(review_bind)):
    clean_text[i]=clean_review(review_bind['bind_text'][i])

define findadj function to find adjectives near a target noun

In [9]:
# define findadj funciton
def findadj(noun,text):
    adj_word = [[1]]*len(text)
    for i in range(len(text)):
        word=text[i]        
        pos_tag = nltk.pos_tag(word)
        adjs = []
        for j in range(len(word)):
            if (word[j] == noun):
                if ((j-1)>=0 and pos_tag[j-1][1] == 'JJ'):
                    adjs.append(pos_tag[j-1][0])
        adj_word[i]=adjs
    return adj_word

## analyse informative nouns

### lobster

In [226]:
# get adjectives near lobster
adj_lobster=findadj('lobster',clean_text)

In [38]:
# prepare empty data frame table
lobster=pd.DataFrame(review_bind['business_id'])
lobster['positive']=[0]*(len(review_bind))
lobster['negative']=[0]*len(review_bind)
lobster['proportion']=[np.nan]*len(review_bind)
lobster.head()

,business_id,positive,negative,proportion
0,nsNONDHbV7Vudqh21uicqw,0,0,NaN
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN
3,vAbRNF0hJt91oCUpm_sdKw,0,0,NaN
4,z8H4Mdzl4jS9pYswj6Jf9w,0,0,NaN


In [39]:
# count positive/negative adjectives near lobster and compute positive proportion
from collections import Counter

for i in range(len(adj_lobster)):
    if(len(adj_lobster[i]) != 0):
        dictionary=Counter(adj_lobster[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    lobster.iat[i,1]=lobster['positive'][i]+dictionary[word]
                else:
                    lobster.iat[i,2]=lobster['negative'][i]+dictionary[word]
        lobster.iat[i,3]=lobster['positive'][i]/(lobster['positive'][i]+lobster['negative'][i])
lobster.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  if sys.path[0] == '':


,business_id,positive,negative,proportion
0,nsNONDHbV7Vudqh21uicqw,88,6,0.93617
1,F06m2yQSPHIrb1IT7heYeQ,3,0,1.00000
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.00000
3,vAbRNF0hJt91oCUpm_sdKw,15,1,0.93750
4,z8H4Mdzl4jS9pYswj6Jf9w,1,0,1.00000


In [41]:
# compute total positive tag
np.sum(lobster['positive'])

6022

In [42]:
# compute total negative tag
np.sum(lobster['negative'])

374

In [46]:
# final table
lobster['stars']=review_bind['stars']
lobster.head()

,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,88,6,0.93617,3.5
1,F06m2yQSPHIrb1IT7heYeQ,3,0,1.00000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.00000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,15,1,0.93750,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,1,0,1.00000,3.5


In [52]:
# compute correlation coeficient
lobster.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.144111
stars,0.144111,1.000000


In [53]:
# save data
lobster.to_csv("lobster.csv",index=False)

## crab

In [225]:
# get adjectives near crab
adj_crab=findadj('crab',clean_text)

In [56]:
# prepare empty data frame table
crab=pd.DataFrame(review_bind['business_id'])
crab['positive']=[0]*(len(review_bind))
crab['negative']=[0]*len(review_bind)
crab['proportion']=[np.nan]*len(review_bind)

In [57]:
# count positive/negative adjectives near crab and compute positive proportion
from collections import Counter

for i in range(len(adj_crab)):
    if(len(adj_crab[i]) != 0):
        dictionary=Counter(adj_crab[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    crab.iat[i,1]=crab['positive'][i]+dictionary[word]
                else:
                    crab.iat[i,2]=crab['negative'][i]+dictionary[word]
        crab.iat[i,3]=crab['positive'][i]/(crab['positive'][i]+crab['negative'][i])

crab['stars']=review_bind['stars']
crab.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  if sys.path[0] == '':


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,57,4,0.934426,3.5
1,F06m2yQSPHIrb1IT7heYeQ,3,1,0.750000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,3,1,0.750000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,0,0,NaN,3.5


In [58]:
# compute total positive/negative tag
print(np.sum(crab['positive']))
print(np.sum(crab['negative']))

3146
479


In [59]:
# compute correlation coeficient
crab.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.141837
stars,0.141837,1.000000


In [60]:
# save data
crab.to_csv("crab.csv",index=False)

## shrimp

In [61]:
# get adjectives near shrimp
adj_shrimp=findadj('shrimp',clean_text)

In [63]:
# prepare empty data frame table
shrimp=pd.DataFrame(review_bind['business_id'])
shrimp['positive']=[0]*(len(review_bind))
shrimp['negative']=[0]*len(review_bind)
shrimp['proportion']=[np.nan]*len(review_bind)

# count positive/negative adjectives near shrimp and compute positive proportion
from collections import Counter

for i in range(len(adj_shrimp)):
    if(len(adj_shrimp[i]) != 0):
        dictionary=Counter(adj_shrimp[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    shrimp.iat[i,1]=shrimp['positive'][i]+dictionary[word]
                else:
                    shrimp.iat[i,2]=shrimp['negative'][i]+dictionary[word]
        shrimp.iat[i,3]=shrimp['positive'][i]/(shrimp['positive'][i]+shrimp['negative'][i])

shrimp['stars']=review_bind['stars']
shrimp.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,42,2,0.954545,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,2,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,0,0,NaN,3.5


In [64]:
# compute total positive/negative tag
print(np.sum(shrimp['positive']))
print(np.sum(shrimp['negative']))

4379
269


In [65]:
# compute correlation coeficient
shrimp.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.040376
stars,0.040376,1.000000


In [66]:
# save data
shrimp.to_csv("shrimp.csv",index=False)

## oyster

In [67]:
# get adjectives near oyster
adj_oyster=findadj('oyster',clean_text)

In [69]:
# prepare empty data frame table
oyster=pd.DataFrame(review_bind['business_id'])
oyster['positive']=[0]*(len(review_bind))
oyster['negative']=[0]*len(review_bind)
oyster['proportion']=[np.nan]*len(review_bind)

# count positive/negative adjectives near oyster and compute positive proportion
from collections import Counter

for i in range(len(adj_oyster)):
    if(len(adj_oyster[i]) != 0):
        dictionary=Counter(adj_oyster[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    oyster.iat[i,1]=oyster['positive'][i]+dictionary[word]
                else:
                    oyster.iat[i,2]=oyster['negative'][i]+dictionary[word]
        oyster.iat[i,3]=oyster['positive'][i]/(oyster['positive'][i]+oyster['negative'][i])

oyster['stars']=review_bind['stars']
oyster.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,41,2,0.953488,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,9,1,0.900000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,1,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,4,1,0.800000,3.5


In [70]:
# compute total positive/negative tag
print(np.sum(oyster['positive']))
print(np.sum(oyster['negative']))

3305
180


In [71]:
# compute correlation coeficient
oyster.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.032166
stars,0.032166,1.000000


In [72]:
# save data
oyster.to_csv("oyster.csv",index=False)

## fish

In [73]:
# get adjectives near fish
adj_fish=findadj('fish',clean_text)

In [75]:
# prepare empty data frame table
fish=pd.DataFrame(review_bind['business_id'])
fish['positive']=[0]*(len(review_bind))
fish['negative']=[0]*len(review_bind)
fish['proportion']=[np.nan]*len(review_bind)

# count positive/negative adjectives near fish and compute positive proportion
from collections import Counter

for i in range(len(adj_fish)):
    if(len(adj_fish[i]) != 0):
        dictionary=Counter(adj_fish[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    fish.iat[i,1]=fish['positive'][i]+dictionary[word]
                else:
                    fish.iat[i,2]=fish['negative'][i]+dictionary[word]
        fish.iat[i,3]=fish['positive'][i]/(fish['positive'][i]+fish['negative'][i])

fish['stars']=review_bind['stars']
fish.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,32,3,0.914286,3.5
1,F06m2yQSPHIrb1IT7heYeQ,7,0,1.000000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,3,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,10,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,19,0,1.000000,3.5


In [76]:
# compute total positive/negative tag
print(np.sum(fish['positive']))
print(np.sum(fish['negative']))

6646
558


In [77]:
# compute correlation coeficient
fish.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.269021
stars,0.269021,1.000000


In [78]:
# save data
fish.to_csv("fish.csv",index=False)

## clam

In [79]:
# get adjectives near clam
adj_clam=findadj('clam',clean_text)

In [81]:
# prepare empty data frame table
clam=pd.DataFrame(review_bind['business_id'])
clam['positive']=[0]*(len(review_bind))
clam['negative']=[0]*len(review_bind)
clam['proportion']=[np.nan]*len(review_bind)

# count positive/negative adjectives near clam and compute positive proportion
from collections import Counter

for i in range(len(adj_clam)):
    if(len(adj_clam[i]) != 0):
        dictionary=Counter(adj_clam[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    clam.iat[i,1]=clam['positive'][i]+dictionary[word]
                else:
                    clam.iat[i,2]=clam['negative'][i]+dictionary[word]
        clam.iat[i,3]=clam['positive'][i]/(clam['positive'][i]+clam['negative'][i])

clam['stars']=review_bind['stars']
clam.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,20,2,0.909091,3.5
1,F06m2yQSPHIrb1IT7heYeQ,4,0,1.000000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,9,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,0,0,NaN,3.5


In [82]:
# compute total positive/negative tag
print(np.sum(clam['positive']))
print(np.sum(clam['negative']))

1154
100


In [83]:
# compute correlation coeficient
clam.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.110865
stars,0.110865,1.000000


In [84]:
# save data
clam.to_csv("clam.csv",index=False)

## waiter

In [103]:
# get adjectives near waiter
adj_waiter=findadj('waiter',clean_text)

In [105]:
# prepare empty data frame table
waiter=pd.DataFrame(review_bind['business_id'])
waiter['positive']=[0]*(len(review_bind))
waiter['negative']=[0]*len(review_bind)
waiter['proportion']=[np.nan]*len(review_bind)

# count positive/negative adjectives near waiter and compute positive proportion
from collections import Counter

for i in range(len(adj_waiter)):
    if(len(adj_waiter[i]) != 0):
        dictionary=Counter(adj_waiter[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    waiter.iat[i,1]=waiter['positive'][i]+dictionary[word]
                else:
                    waiter.iat[i,2]=waiter['negative'][i]+dictionary[word]
        waiter.iat[i,3]=waiter['positive'][i]/(waiter['positive'][i]+waiter['negative'][i])

waiter['stars']=review_bind['stars']
waiter.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,12,1,0.923077,3.5
1,F06m2yQSPHIrb1IT7heYeQ,1,0,1.000000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,7,3,0.700000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,2,2,0.500000,3.5


In [106]:
# compute total positive/negative tag
print(np.sum(waiter['positive']))
print(np.sum(waiter['negative']))

1468
267


In [107]:
# compute correlation coeficient
waiter.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.267945
stars,0.267945,1.000000


In [108]:
# save data
waiter.to_csv("waiter.csv",index=False)

## waitress

In [109]:
# get adjectives near waitress
adj_waitress=findadj('waitress',clean_text)

In [111]:
# prepare empty data frame table
waitress=pd.DataFrame(review_bind['business_id'])
waitress['positive']=[0]*(len(review_bind))
waitress['negative']=[0]*len(review_bind)
waitress['proportion']=[np.nan]*len(review_bind)

# count positive/negative adjectives near waitress and compute positive proportion
from collections import Counter

for i in range(len(adj_waitress)):
    if(len(adj_waitress[i]) != 0):
        dictionary=Counter(adj_waitress[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    waitress.iat[i,1]=waitress['positive'][i]+dictionary[word]
                else:
                    waitress.iat[i,2]=waitress['negative'][i]+dictionary[word]
        waitress.iat[i,3]=waitress['positive'][i]/(waitress['positive'][i]+waitress['negative'][i])

waitress['stars']=review_bind['stars']
waitress.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,9,2,0.818182,3.5
1,F06m2yQSPHIrb1IT7heYeQ,3,1,0.750000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,1,0,1.000000,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,4,0,1.000000,3.5


In [112]:
# compute total positive/negative tag
print(np.sum(waitress['positive']))
print(np.sum(waitress['negative']))

1424
284


In [113]:
# compute correlation coeficient
waitress.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.164274
stars,0.164274,1.000000


In [114]:
# save data
waitress.to_csv("waitress.csv",index=False)

## chef

In [115]:
# get adjectives near chef
adj_chef=findadj('chef',clean_text)

In [117]:
# prepare empty data frame table
chef=pd.DataFrame(review_bind['business_id'])
chef['positive']=[0]*(len(review_bind))
chef['negative']=[0]*len(review_bind)
chef['proportion']=[np.nan]*len(review_bind)

# count positive/negative adjectives near chef and compute positive proportion
from collections import Counter

for i in range(len(adj_chef)):
    if(len(adj_chef[i]) != 0):
        dictionary=Counter(adj_chef[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    chef.iat[i,1]=chef['positive'][i]+dictionary[word]
                else:
                    chef.iat[i,2]=chef['negative'][i]+dictionary[word]
        chef.iat[i,3]=chef['positive'][i]/(chef['positive'][i]+chef['negative'][i])

chef['stars']=review_bind['stars']
chef.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,2,0,1.0,3.5
1,F06m2yQSPHIrb1IT7heYeQ,1,0,1.0,4.0
2,c_FTil8s5PS2l_YJDQAXhA,0,0,NaN,4.0
3,vAbRNF0hJt91oCUpm_sdKw,1,0,1.0,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,3,0,1.0,3.5


In [118]:
# compute total positive/negative tag
print(np.sum(chef['positive']))
print(np.sum(chef['negative']))

818
48


In [119]:
# compute correlation coeficient
chef.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.252342
stars,0.252342,1.000000


In [120]:
# save data
chef.to_csv("chef.csv",index=False)

## manager

In [121]:
# get adjectives near manager
adj_manager=findadj('manager',clean_text)

In [123]:
# prepare empty data frame table
manager=pd.DataFrame(review_bind['business_id'])
manager['positive']=[0]*(len(review_bind))
manager['negative']=[0]*len(review_bind)
manager['proportion']=[np.nan]*len(review_bind)

# count positive/negative adjectives near manager and compute positive proportion
from collections import Counter

for i in range(len(adj_manager)):
    if(len(adj_manager[i]) != 0):
        dictionary=Counter(adj_manager[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    manager.iat[i,1]=manager['positive'][i]+dictionary[word]
                else:
                    manager.iat[i,2]=manager['negative'][i]+dictionary[word]
        manager.iat[i,3]=manager['positive'][i]/(manager['positive'][i]+manager['negative'][i])

manager['stars']=review_bind['stars']
manager.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,2,2,0.5,3.5
1,F06m2yQSPHIrb1IT7heYeQ,0,0,NaN,4.0
2,c_FTil8s5PS2l_YJDQAXhA,1,0,1.0,4.0
3,vAbRNF0hJt91oCUpm_sdKw,0,0,NaN,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,1,0,1.0,3.5


In [124]:
# compute total positive/negative tag
print(np.sum(manager['positive']))
print(np.sum(manager['negative']))

586
263


In [125]:
# compute correlation coeficient
manager.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.209061
stars,0.209061,1.000000


In [126]:
# save data
manager.to_csv("manager.csv",index=False)

## service

In [151]:
# get adjectives near service
adj_service=findadj('service',clean_text)

In [153]:
# prepare empty data frame table
service=pd.DataFrame(review_bind['business_id'])
service['positive']=[0]*(len(review_bind))
service['negative']=[0]*len(review_bind)
service['proportion']=[np.nan]*len(review_bind)

# count positive/negative adjectives near service and compute positive proportion
from collections import Counter

for i in range(len(adj_service)):
    if(len(adj_service[i]) != 0):
        dictionary=Counter(adj_service[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    service.iat[i,1]=service['positive'][i]+dictionary[word]
                else:
                    service.iat[i,2]=service['negative'][i]+dictionary[word]
        service.iat[i,3]=service['positive'][i]/(service['positive'][i]+service['negative'][i])

service['stars']=review_bind['stars']
service.head()

,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,94,21,0.817391,3.5
1,F06m2yQSPHIrb1IT7heYeQ,13,3,0.812500,4.0
2,c_FTil8s5PS2l_YJDQAXhA,13,1,0.928571,4.0
3,vAbRNF0hJt91oCUpm_sdKw,20,7,0.740741,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,34,2,0.944444,3.5


In [154]:
# compute total positive/negative tag
print(np.sum(service['positive']))
print(np.sum(service['negative']))

15458
3162


In [155]:
# compute correlation coeficient
service.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.580465
stars,0.580465,1.000000


In [156]:
# save data
service.to_csv("service.csv",index=False)

## price

In [157]:
# get adjectives near price
adj_price=findadj('price',clean_text)

In [159]:
# prepare empty data frame table
price=pd.DataFrame(review_bind['business_id'])
price['positive']=[0]*(len(review_bind))
price['negative']=[0]*len(review_bind)
price['proportion']=[np.nan]*len(review_bind)

# count positive/negative adjectives near price and compute positive proportion
from collections import Counter

for i in range(len(adj_price)):
    if(len(adj_price[i]) != 0):
        dictionary=Counter(adj_price[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    price.iat[i,1]=price['positive'][i]+dictionary[word]
                else:
                    price.iat[i,2]=price['negative'][i]+dictionary[word]
        price.iat[i,3]=price['positive'][i]/(price['positive'][i]+price['negative'][i])

price['stars']=review_bind['stars']
price.head()

C:\Users\GAO ZEJIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in longlong_scalars


,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,32,5,0.864865,3.5
1,F06m2yQSPHIrb1IT7heYeQ,11,0,1.000000,4.0
2,c_FTil8s5PS2l_YJDQAXhA,5,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,9,2,0.818182,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,13,2,0.866667,3.5


In [160]:
# compute total positive/negative tag
print(np.sum(price['positive']))
print(np.sum(price['negative']))

7869
802


In [161]:
# compute correlation coeficient
price.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.221228
stars,0.221228,1.000000


In [162]:
# save data
price.to_csv("price.csv",index=False)

## food

In [219]:
# get adjectives near food
adj_food=findadj('food',clean_text)

In [221]:
# prepare empty data frame table
food=pd.DataFrame(review_bind['business_id'])
food['positive']=[0]*(len(review_bind))
food['negative']=[0]*len(review_bind)
food['proportion']=[np.nan]*len(review_bind)

# count positive/negative adjectives near food and compute positive proportion
from collections import Counter

for i in range(len(adj_food)):
    if(len(adj_food[i]) != 0):
        dictionary=Counter(adj_food[i])
        for word in dictionary.keys():
            if (word in dict_adj.keys()):
                if (dict_adj[word] == 'positive'):
                    food.iat[i,1]=food['positive'][i]+dictionary[word]
                else:
                    food.iat[i,2]=food['negative'][i]+dictionary[word]
        food.iat[i,3]=food['positive'][i]/(food['positive'][i]+food['negative'][i])

food['stars']=review_bind['stars']
food.head()

,business_id,positive,negative,proportion,stars
0,nsNONDHbV7Vudqh21uicqw,102,15,0.871795,3.5
1,F06m2yQSPHIrb1IT7heYeQ,70,1,0.985915,4.0
2,c_FTil8s5PS2l_YJDQAXhA,9,0,1.000000,4.0
3,vAbRNF0hJt91oCUpm_sdKw,13,1,0.928571,3.5
4,z8H4Mdzl4jS9pYswj6Jf9w,54,4,0.931034,3.5


In [222]:
# compute total positive/negative tag
print(np.sum(food['positive']))
print(np.sum(food['negative']))

24047
1907


In [223]:
# compute correlation coeficient
food.loc[:,['proportion','stars']].corr()

,proportion,stars
proportion,1.000000,0.547101
stars,0.547101,1.000000


In [224]:
# save data
food.to_csv("food.csv",index=False)